In [2]:
import cv2
import time
import pandas as pd
from ultralytics import YOLO

In [10]:
def detect_and_log_speed(video_path, model_paths, output_csv="detection_speed.csv"):
    """
    Detects objects in a video using different YOLO models and logs the detection speed (in ms) for each frame.
    Then calculates the average detection speed for each model and logs the result to a CSV.
    
    :param video_path: Path to the input video file.
    :param model_paths: List of paths to the YOLO model files.
    :param output_csv: Path to the output CSV file for logging results.
    """
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video file: {video_path}")
        return

    # Prepare the CSV headers
    headers = ["Model", "Average Inference Time (ms)"]
    log_data = []

    # Loop over each model
    for model_path in model_paths:
        model_name = model_path.split("/")[-1].split(".")[0]  # Extract model name from path
        model = YOLO(model_path)

        total_inference_time = 0
        frame_count = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame_count += 1

            # Record the start time
            start_time = time.time()

            # Run detection on the frame
            results = model(frame, verbose=False)[0]

            # Calculate inference time in milliseconds
            inference_time = (time.time() - start_time) * 1000  # Convert to milliseconds
            total_inference_time += inference_time

            # Optionally, display the frame with bounding boxes (you can remove this in production)
            for det in results.boxes:
                x1, y1, x2, y2 = map(int, det.xyxy[0])
                label = results.names[int(det.cls[0])]
                confidence = float(det.conf[0])
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                text = f"{label} {confidence:.2f}"
                cv2.putText(frame, text, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Calculate average inference time for this model
        average_inference_time = total_inference_time / frame_count if frame_count > 0 else 0

        # Log the average inference time for this model
        log_data.append([model_name, average_inference_time])

        # Reset video capture to the beginning for the next model
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

    # Save the log data to CSV
    df = pd.DataFrame(log_data, columns=headers)
    df.to_csv(output_csv, index=False)
    print(f"Speed log saved to {output_csv}")

    cap.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    # List of model file paths (you can change these as per your models)
    model_paths = [
        # "yolov5nu.pt",  # Lightest model
        # "yolov5su.pt",  # Medium model
        # "yolov5xu.pt",  # Heavier model
        # "yolov8n.pt",  # YOLOv8 Nano model
        # "yolov9t.pt",  # YOLOv9 Tiny model
        "yolo11n.pt",  # YOLOv11 Nano model
        "yolo11s.pt",  # YOLOv11 Small model
    ]
    
    # Input video path
    video_path = "WIN_20250401_09_02_07_Pro.mp4"  # Replace with your video file path

    # Run the speed detection and logging function
    detect_and_log_speed(video_path, model_paths)


Speed log saved to detection_speed.csv
